In [61]:
import os
import sys
import glob
import obspy
import subprocess
from obspy.core.utcdatetime import UTCDateTime
from obspy.taup import TauPyModel
from obspy.signal.rotate import rotate_ne_rt
import numpy as np

# first operation in processing seis data
def seis_data_rename(dirname, fnames):
    if os.path.exists(dirname):
        os.chdir(dirname)
    else:
        sys.exit('No such dir: %s!' % dirname)
    for fname in glob.glob(fnames):
        tr = obspy.read(fname)[0]
        # derive the year, juldayday, hour, minute, second, microsecond
        f_utc = tr.stats.starttime
        # int
        year, julday, hour = f_utc.year, f_utc.julday, f_utc.hour
        minute, second = f_utc.minute, f_utc.second
        microsecond = f_utc.microsecond
        # get network, station, location, channel
        # str
        knetwk, kstnm = tr.stats.network, tr.stats.station
        khole, kcmpnm = tr.stats.location, tr.stats.channel
        # give old file a new name
        newfname = '%d.%d.%d.%d.%d.%d.%s.%s.%s.%s.SAC' \
        % (year, julday, hour, minute, second, microsecond, 
           knetwk, kstnm, khole, kcmpnm)
        os.rename(fname, newfname)
    
    # instrument file corresponding to SAC file
    for fname in glob.glob('SACPZ*'):
        knetwk, kstnm, khole, kcmpnm = fname.split('.')[1:5]
        if khole == '--':
            newfname = 'SAC_PZs_%s_%s__%s' % (knetwk, kstnm, kcmpnm)
        else:
            newfname = 'SAC_PZs_%s_%s_%s_%s' % (knetwk, kstnm, khole, kcmpnm)
        os.rename(fname, newfname)
    
    # unfortunately, have to unlink the SAC without corresponding instrument
    for fname in glob.glob(fnames):
        knetwk, kstnm, khole, kcmpnm = fname.split('.')[6:10]
        pzfname = 'SAC_PZs_%s_%s_%s_%s' % (knetwk, kstnm, khole, kcmpnm)
        if not os.path.exists(pzfname):
            print('*.%s.%s.%s.%s.SAC without corresponding instrument!' 
                  % (knetwk, kstnm, khole, kcmpnm))
            os.unlink(fname)
    return

# second operation in processing seis data
def add_arrival_info(fname, phase_list=('PKIKP',), tn_dict={'PKIKP':'t4'}):
    """
    Now, P and S travel time are added in seismic data.
    
    As for other phase, can be added via phase_list and tn_dict.
    """
    # read seismic data and get evdp, gcarc
    tr = obspy.read(fname)[0]
    gc = tr.stats.sac['gcarc'] # np.float32
    evdp = tr.stats.sac['evdp'] # np.float32
    
    model = TauPyModel('prem')
    # derive P wave arrival time
    arrivals = model.get_travel_times(evdp, gc, phase_list=('P',))
    try:
        arrival = arrivals[0]
    except IndexError:
        P_travel = ''
    else:
        P_travel = arrival.time # np.float32
    # store the P travel in SAC
    tr.stats.sac['t1'] = P_travel
    tr.stats.sac['kt1'] = 'P'
    
    # derive S or Sdiff wave arrival time
    arrivals = model.get_travel_times(evdp, gc, phase_list=('S', 'Sdiff'))
    try:
        arrival = arrivals[0]
    except IndexError:
        S_travel = ''
    else:
        S_travel = arrival.time # np.float32
        S_name = arrival.name # str
    # store the P travel in SAC
    tr.stats.sac['t2'] = S_travel
    tr.stats.sac['kt2'] = S_name
    
    # add other phase travel time
    for phase in phase_list:
        arrivals = model.get_travel_times(evdp, gc, phase_list=(phase,))
        try:
            arrival = arrivals[0]
        except IndexError:
            phase_travel = ''
        else:
            phase_travel = arrival.time # np.float32
            phase_name = arrival.name # str
            
            tn = tn_dict[phase_name]
            ktn = 'k%s' % tn
            tr.stats.sac[ktn] = phase_name
            # store phase travel
            tr.stats.sac[tn] = phase_travel
    
    tr.write(fname)
    
    return

def get_o_from_cmt(fname='../CMTSOLUTION'):
    if not os.path.exists(fname):
        sys.exit('No such file: %s!' % fname)
    with open(fname, 'r') as fo:
        # event time info
        oyear, omonth, oday, ohour, ominute, osecond \
        = fo.readline().rstrip().split()[0:6]
        oyear = oyear[4:8]
        osecond, omicrosecond = osecond.split('.')
        
        # source location info
        fo.readline()
        time_shift = fo.readline().rstrip().split()[-1]
        half_duration = fo.readline().rstrip().split()[-1]
        evla = fo.readline().rstrip().split()[-1]
        evlo = fo.readline().rstrip().split()[-1]
        evdp = fo.readline().rstrip().split()[-1]
        
    return int(oyear), int(omonth), int(oday), int(ohour), \
int(ominute), int(osecond), int(omicrosecond), evla, evlo, evdp

def get_pz_info(pzfname):
    with open(pzfname, 'r') as fo:
        for i in range(19):
            fo.readline()
        # get gain
        gain = fo.readline().rstrip().split(':')[1]
        gain = float(gain.split()[0])
        fo.readline()
        # get sensitivity
        sens = fo.readline().rstrip().split(':')[1]
        sens = float(sens.split()[0])
        for i in range(3):
            fo.readline()
        # get zeros
        zeros = []
        while True:
            re, im = fo.readline().split()
            if re == 'POLES':
                break
            re, im = float(re), float(im)
            zeros.append(complex(re, im))
        # get poles
        poles = []
        while True:
            re, im = fo.readline().split()
            if re == 'CONSTANT':
                break
            re, im = float(re), float(im)
            poles.append(complex(re, im))
    
    return gain, sens, zeros, poles
### 上述自定义获取PZ格式仪器响应的函数，实属无奈之举，obspy中没找到对应方法
    

def seis_data_process(dirname, fnames, delta=0.025, o_left=0, 
                      o_right=3500, freqmin=0.02, freqmax=1):
    if os.path.exists(dirname):
        os.chdir(dirname)
    else:
        sys.exit('No such dir: %s!' % dirname)
    for fname in glob.glob(fnames):
        tr = obspy.read(fname)[0]
        # resample seismic data
        delta0 = tr.stats.delta # float
        if delta > delta0: # downsampling
            # low pass, unsure parameters
            freq = 0.5 / delta
            tr.filter('lowpass', freq=freq, zerophase=True)
        sample_rate = 1 / delta
        tr.interpolate(sampling_rate=sample_rate)

        # add event info
        # derive begin time of event
        oyear, omonth, oday, ohour, ominute, osecond, omicrosecond, \
        evla, evlo, evdp = get_o_from_cmt()
        # derive referent time
        reft = UTCDateTime(year=tr.stats.sac['nzyear'], 
                           julday=tr.stats.sac['nzjday'], 
                           hour=tr.stats.sac['nzhour'], 
                           minute=tr.stats.sac['nzmin'], 
                           second=tr.stats.sac['nzsec'], 
                           microsecond=tr.stats.sac['nzmsec'])
        # substract o-deviation all time variations have been defined before
        o_reft = UTCDateTime(oyear, omonth, oday, ohour, ominute, 
                             osecond, omicrosecond) - reft
        o = UTCDateTime(oyear, omonth, oday, ohour, ominute, 
                        osecond, omicrosecond)
        
        tr.stats.sac['b'] = tr.stats.sac['b'] - o_reft
        tr.stats.sac['e'] = tr.stats.sac['e'] - o_reft
        tr.stats.sac['o'] = 0
        tr.stats.sac['nzyear'] = o.year
        tr.stats.sac['nzjday'] = o.julday
        tr.stats.sac['nzhour'] = o.hour
        tr.stats.sac['nzmin'] = o.minute
        tr.stats.sac['nzsec'] = o.second
        tr.stats.sac['nzmsec'] = o.microsecond
### 上述手动处理时间，不知会不会出现问题，对于处理各种各样的地震数据

        tr.write(fname)
        tr = obspy.read(fname)[0]

        # cut seismic data
        if o_left > tr.stats.sac['b']:
            start_t = o_left
        else:
            start_t = tr.stats.sac['b']
        if o_right < tr.stats.sac['e']:
            end_t = o_right
        else:
            end_t = tr.stats.sac['e']
        tr.trim(o + start_t, o + end_t)
        
        # rmean, rtrend, tapper
        tr.detrend('constant')
        tr.detrend(type='linear')
        tr.taper(max_percentage=0.05)
        
        # filter (unsure parameters)
        tr.filter('bandpass', freqmin=freqmin, freqmax=freqmax, 
                  zerophase=True)
        
        # transfer instrument responce
        pzfname = 'SAC_PZs_%s' % ('_'.join(fname.split('.')[6:10]))
        gain, sens, zeros, poles = get_pz_info(pzfname)
        paz_info = {'gain' : gain, 'poles' : poles, 
                      'sensitivity' : sens, 'zeros' : zeros}
        f2, f3 = freqmin, freqmax
        f1, f4 = f2*0.8, f3*1.2
        tr.simulate(paz_remove=paz_info, pre_filt=[f1, f2, f3, f4], 
                    water_level=600.0)
### according to SAC, there should be an operation "mul 1.0e9"
        
        tr.write(fname)
        
        add_arrival_info(fname=fname)
    
    return

def seis_data_rotate(dirname, fnames):
    if os.path.exists(dirname):
        os.chdir(dirname)
    else:
        sys.exit('No such dir: %s!' % dirname)
    
    prefixs = set()
    for fname in glob.glob(fnames):
        prefix = '.'.join(fname.split('.')[0:9])
        prefixs.add(prefix)
    
    # event time
    oyear, omonth, oday, ohour, ominute, osecond, omicrosecond, \
        evla, evlo, evdp = get_o_from_cmt()
    o = UTCDateTime(oyear, omonth, oday, ohour, ominute, 
                        osecond, omicrosecond)
    
    for prefix in prefixs:
        # Check BHZ,BHE.BHN and the orthogonality of BHE,BHN
        BHZ = prefix + '.BHZ.SAC'
        if not os.path.exists(BHZ):
            print("%s: Vertical component missing!" % prefix)
            continue
        if os.path.exists(prefix + '.BHE.SAC') \
        and os.path.exists(prefix + '.BHN.SAC'):
            BHE = prefix + '.BHE.SAC'
            BHN = prefix + '.BHN.SAC'
        elif os.path.exists(prefix + '.BH1.SAC') \
        and os.path.exists(prefix + '.BH2.SAC'):
            BHE = prefix + '.BH1.SAC'
            BHN = prefix + '.BH2.SAC'
        else:
            print("%s: Horizontal component missing!" % prefix)
            continue
        # check whether E and N are orthogonal
        tre = obspy.read(BHE)[0]
        ecmpaz = tre.stats.sac['cmpaz'] # np.float32
        trn = obspy.read(BHN)[0]
        ncmpaz = trn.stats.sac['cmpaz'] # np.float32
        cmpaz_delta = abs(ecmpaz - ncmpaz)
        if not (abs(cmpaz_delta-90) <= 0.01 or abs(cmpaz_delta-270) <= 0.01):
            print('%s: cmpaz1=%.3f, cmpaz2=%.3f are not orthogonal!' \
                  % (prefix, ecmpaz, ncmpaz))
            continue
        # chack b, e, delta
        trz = obspy.read(BHZ)[0]
        zb, ze = trz.stats.sac['b'], trz.stats.sac['e'] # np.float32
        zdelta = trz.stats.delta # float
        eb, ee = tre.stats.sac['b'], tre.stats.sac['e']
        edelta = tre.stats.delta
        nb, ne = trn.stats.sac['b'], trn.stats.sac['e']
        ndelta = trn.stats.delta
        if not (zdelta == edelta) and (zdelta == ndelta):
            print("%s: delta not equal!" % prefix)
            continue
        # unify cut window
        begin = max(zb, eb, nb)
        end = min(ze, ee, ne)
        
        # outfile
        BHR, BHT, BHZ0 = prefix + '.BHR', prefix + '.BHT', prefix + '.BHZ'
        tre.trim(o + begin, o + end)
        tre.write(BHE)
        trn.trim(o + begin, o + end)
        trn.write(BHN)
        trz.trim(o + begin, o + end)
        trz.write(BHZ0, format='SAC')
        
        tre = obspy.read(BHE)[0]
        ebaz = tre.stats.sac['baz']
        trn = obspy.read(BHN)[0]
        nbaz = trn.stats.sac['baz']
        if not (ebaz == nbaz):
            print("%s: baz not equal!" % prefix)
        trn.data, tre.data = rotate_ne_rt(trn.data, tre.data, ebaz)
        trn.stats.channel, tre.stats.channel = 'BHR', 'BHT'
        trn.write(BHR, format='SAC')
        tre.write(BHT, format='SAC')
    
    return
            